### This code will get Station ID, Rainfall Measurements, and Coordinates from English and Scottish Rain Gauges and writes them on a CSV

In [ ]:
import requests
import csv

# Function to fetch station data with coordinates for England
def fetch_station_data_eng():
    """
    Fetches station data including latitude and longitude for England.
    Makes a request to the England-specific API and processes the JSON response.
    Returns a dictionary with station notations as keys and coordinates as values.
    """
    url = 'https://environment.data.gov.uk/flood-monitoring/id/stations?parameter=rainfall'
    response = requests.get(url)
    eng_station_data = {}
    if response.status_code == 200:
        data = response.json()
        for station in data['items']:
            eng_station_data[station.get('notation')] = {
                'latitude': station.get('lat'),
                'longitude': station.get('long')
            }
    return eng_station_data

# Function to fetch rainfall measurements for England
def get_rainfall_data_eng():
    """
    Retrieves rainfall measurement data for England.
    Sends a request to the England-specific rainfall data API and parses the JSON response.
    Returns a list of rainfall measurement items.
    """
    eng_url = "http://environment.data.gov.uk/flood-monitoring/id/measures?parameter=rainfall"
    eng_response = requests.get(eng_url)
    if eng_response.status_code == 200:
        eng_data = eng_response.json()
        return eng_data['items']
    return []

# Function to fetch station data with coordinates for Scotland
def fetch_station_data_sco():
    """
    Fetches station data including latitude and longitude for Scotland.
    Makes a request to the Scotland-specific API and processes the JSON response.
    Returns a dictionary with station numbers as keys and coordinates as values.
    """
    sco_url = "https://www2.sepa.org.uk/rainfall/api/Stations"
    sco_response = requests.get(sco_url)
    sco_station_data = {}
    if sco_response.status_code == 200:
        sco_stations = sco_response.json()
        for station in sco_stations:
            sco_station_data[station['station_no']] = {
                'latitude': station.get('station_latitude'),
                'longitude': station.get('station_longitude')
            }
    return sco_station_data

# Function to fetch latest hourly rainfall data for Scotland
def get_rainfall_data_sco(station_id):
    """
    Retrieves the latest hourly rainfall data for a specific station in Scotland.
    Sends a request to the Scotland-specific hourly rainfall data API for a given station.
    Returns the latest rainfall data item if available.
    """
    sco_url = f"https://www2.sepa.org.uk/rainfall/api/Hourly/{station_id}?all=true"
    sco_response = requests.get(sco_url)
    if sco_response.status_code == 200 and sco_response.json():
        return sco_response.json()[-1]
    return None

# Fetching and processing data
eng_station_coordinates = fetch_station_data_eng()
eng_rainfall_data = get_rainfall_data_eng()
sco_station_coordinates = fetch_station_data_sco()
sco_rainfall_data = {station_id: get_rainfall_data_sco(station_id) for station_id in sco_station_coordinates}

# Combine the data
combined_data = []

# Process and combine England data
for measurement in eng_rainfall_data:
    station_id = measurement.get('stationReference')
    rainfall = measurement.get('latestReading', {}).get('value')
    coordinates = eng_station_coordinates.get(station_id, {'latitude': None, 'longitude': None})
    combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'England'])

# Process and combine Scotland data
for station_id, latest_hourly_data in sco_rainfall_data.items():
    if latest_hourly_data:
        rainfall = latest_hourly_data['Value']
        coordinates = sco_station_coordinates[station_id]
        combined_data.append([station_id, rainfall, coordinates['latitude'], coordinates['longitude'], 'Scotland'])

# Write to CSV
filename = "rainfall_measurements.csv"
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Station ID", "Rainfall Measurement (mm)", "Latitude", "Longitude", "Country"])
    writer.writerows(combined_data)

print(f"Data has been written to {filename}")
